In [1]:
from unstructured.partition import pdf, docx, pptx
from langchain_core.documents import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import unstructured
import os

c:\Users\parth\anaconda3\envs\cs246helper\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('api_keys.txt') as f: api_keys = dict(key.split() for key in f.read().split('\n'))
api_keys['unstructured_api_key']

'RYrkDfIfLO1cb4Ltyaslqe10iJPQ01'

In [3]:
dir = 'docs/cs246'

In [11]:
text = ''
interim_dir = f'{dir}/interim'
os.mkdir(interim_dir)


for filename in os.listdir(dir):
    filepath = f'{dir}/{filename}'
    if os.path.isfile(filepath) and filename.split('.')[-1] == 'pdf':
        print(f'processing {filename}')
        elements = pdf.partition_pdf(
            filename=filepath,
            strategy='hi_res',
            extract_images_in_pdf=True,
            extract_image_block_types=['Image', 'Table'],
            extract_image_block_output_dir=interim_dir,
            infer_table_structure=True,
            max_characters=1000,
            new_after_n_chars=1500,
            combine_text_under_n_chars=250
            )
        for element in elements:
            match element.category:
                case 'Image': pass
                case 'Table': pass
                case 'Formula': pass
                case _: text += f'{element.text}\n'

 

processing notes-197-204.pdf


In [13]:
print(text)

Classes
We know that we can keep related pieces of information together in C/C++ by putting them into a structure, using the struct keyword. This improves cohesion. There is, however, another way that C++ lets you improve cohesion that cannot be done in C; we can also put functions inside of a structure. This lets us keep related data and functions (now called operations or methods or member functions) in the same data structure, which is called a class. (We'll see the C++ class keyword shortly, in The class Keyword. For now, we're just going to use the struct keyword since it provides similar functionality.) A class is a type. An instance of a class is called an object.
Let's take a look at an example:
File: student.h Explanation #ifndef _STUDENT_H_ #define _STUDENT_H_ struct Student { int assns, mt, final; float grade(); }; #endif Here we can see that the Student class has three data elds used to hold marks respectively for assignments (assns), the midterm (mt), and the nal exam (fin

In [13]:
all_text = ''
for each pdf:
    elements = partition_pdf(pdf)
    text = ""
    for element in elements:
        if element is image:
            img = open(image)
            chart = llm_is_chart(img)
            if chart: text += deplot(img)
            else: text += llm(img)
        elif element is formula:
            
        else: text += element.text


# image?
# formula?
# else:

SyntaxError: invalid syntax (1053523969.py, line 1)

In [10]:
len(elements)

16

In [11]:
for element in elements:
    print(type(element))
    print(element)
    print('\n'*5)

<class 'unstructured.documents.elements.CompositeElement'>
Classes

We know that we can keep related pieces of information together in C/C++ by putting them into a structure, using the struct keyword. This improves cohesion. There is, however, another way that C++ lets you improve cohesion that cannot be done in C; we can also put functions inside of a structure. This lets us keep related data and functions (now called operations or methods or member functions) in the same data structure, which is called a class. (We'll see the C++ class keyword shortly, in The class Keyword. For now, we're just going to use the struct keyword since it provides similar functionality.) A class is a type. An instance of a class is called an object.

Let's take a look at an example:






<class 'unstructured.documents.elements.TableChunk'>
File: student.h Explanation #ifndef _STUDENT_H_ #define _STUDENT_H_ struct Student { int assns, mt, final; float grade(); }; #endif Here we can see that the Student cl

In [7]:
len(elements)

45

In [6]:
for element in elements:
    print(type(element))
    print(element)
    print('\n'*5)

<class 'unstructured.documents.elements.Title'>
Classes






<class 'unstructured.documents.elements.NarrativeText'>
We know that we can keep related pieces of information together in C/C++ by putting them into a structure, using the struct keyword. This improves cohesion. There is, however, another way that C++ lets you improve cohesion that cannot be done in C; we can also put functions inside of a structure. This lets us keep related data and functions (now called operations or methods or member functions) in the same data structure, which is called a class. (We'll see the C++ class keyword shortly, in The class Keyword. For now, we're just going to use the struct keyword since it provides similar functionality.) A class is a type. An instance of a class is called an object.






<class 'unstructured.documents.elements.NarrativeText'>
Let's take a look at an example:






<class 'unstructured.documents.elements.Table'>
File: student.h Explanation #ifndef _STUDENT_H_ #define _STUD

In [53]:
for element in elements:
    if isinstance(element, unstructured.documents.elements.Table): 
        print(element.text)
        break

File: student.h Explanation #ifndef _STUDENT_H_ #define _STUDENT_H_ struct Student { int assns, mt, final; float grade(); }; #endif Here we can see that the Student class has three data elds used to hold marks respectively for assignments (assns), the midterm (mt), and the nal exam (final). It has a single method, grade, that takes no parameters and returns the nal grade in the course. In general, we want to try and avoid giving the client/user too much information about our class and its functionality, which is one of the reasons we only give the function signature in the header le. Ideally, we'd give the client our header le and the student.o le, which they could then link in with their code, and they'd never need to know how the nal course grade was actually calculated! File: student.cc Explanation #include "student.h" Note that in the implementation le for the Student, we


In [49]:
for element in elements:
    if isinstance(element, unstructured.documents.elements.Table): 
        print(element.metadata.to_dict()['text_as_html'])
        break

<table><thead><tr><th>File: student .h</th><th>Explanation</th></tr></thead><tbody><tr><td>i#define _STUDENT_H_</td><td>Here we can see that the Student class has three data fields used to hold marks respectively for assignments (assns), the</td></tr><tr><td>int assns, mt, final;</td><td>method, grade, that takes no parameters and returns the</td></tr><tr><td>float grade();</td><td>final grade in the course. In general, we want to try and avoid</td></tr><tr><td>H#endif</td><td>and its functionality, which is one of the reasons we only give</td></tr><tr><td>33 File: student.cc</td><td>giving the client/user too much information about our class the function signature in the header file. Ideally, we'd give the client our header file and the student. file, which they could then link in with their code, and they'd never need to know how the final course grade was actually calculated! Explanation</td></tr><tr><td></td><td></td></tr><tr><td></td><td></td></tr><tr><td>#include "student.h"</td>

In [50]:
for element in elements:
    if isinstance(element, unstructured.documents.elements.Table): 
        display(HTML(element.metadata.to_dict()['text_as_html']))
        break

File: student .h,Explanation
i#define _STUDENT_H_,"Here we can see that the Student class has three data fields used to hold marks respectively for assignments (assns), the"
"int assns, mt, final;","method, grade, that takes no parameters and returns the"
float grade();,"final grade in the course. In general, we want to try and avoid"
H#endif,"and its functionality, which is one of the reasons we only give"
33 File: student.cc,"giving the client/user too much information about our class the function signature in the header file. Ideally, we'd give the client our header file and the student. file, which they could then link in with their code, and they'd never need to know how the final course grade was actually calculated! Explanation"
,
,
"#include ""student.h""","Note that in the implementation file for the Student, we"


In [6]:
documents = []
for element in elements:
    if isinstance(element, unstructured.documents.elements.CompositeElement): documents.append(Document(page_content=element.text, metadata=element.metadata.to_dict()))
    elif isinstance(element, unstructured.documents.elements.Table): pass # do something here

db = FAISS.from_documents(documents, HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5'))
retriever = db.as_retriever(search_type="similarity", search_kwargs={'k': 4})

c:\Users\parth\anaconda3\envs\cs246helper\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\parth\anaconda3\envs\cs246helper\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_hub_download` instead.
  warnings.warn(warning_message, FutureWarning)
You try to use a model that was created with version 2.2.2, however, your version is 2.2.1. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [7]:
!nougat 138_midterm.pdf -o output/

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.12 (you have 1.4.11). Upgrade using: pip install --upgrade albumentations
INFO:root:Output directory does not exist. Creating output directory.
C:\Users\parth\anaconda3\envs\cs246helper\Lib\site-packages\torch\functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3610.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

  0%|          | 0/2 [00:00<?, ?it/s]C:\Users\parth\anaconda3\envs\cs246helper\Lib\site-packages\nougat\model.py:437: UserWarning: var(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\Re